In [8]:
import numpy as np                  # array manipulation
import astropy.io.fits as fits      # fits file manipulation
import glob, os                     # to grab source files

os.chdir("C:/Projects/varstarp/NGC2180/20161107")

files = glob.glob("*.fts")       # list of all source files
targ_ra = 128.86346                 # right ascension of target
targ_dec = 19.75875                 # declination of target
match_r = 2.0                       # source matching radius in arcesc

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Projects/varstarp/NGC2180/20161107'

In [ ]:
Nims = len(files)
HJD = np.empty(Nims)                # store HJD of observation
mag = np.empty(Nims)                # store the magnitude
magunc = np.empty(Nims)             # store the mag uncertainty

# Now loop over the files to create the light curve
for imnum, cat in enumerate(files):
    HJD[imnum] = fits.getval(cat, "HJD", 2)     # HJD stored in cat header
    # vvv New condition is added here vvv
    if ( fits.getval(cat, "PHTCALFL", 2) == 1 and 
        fits.getval(cat, "PCALRMSE", 2) < 0.04 ):  # phot. conditions?
        Sdat = fits.getdata(cat)
        Sdat_RA = Sdat["ALPHAWIN_J2000"]            # RA list for cat
        Sdat_Dec = Sdat["DELTAWIN_J2000"]           # Dec list for cat
        source_sep = np.hypot( 
        (Sdat_RA - targ_ra) * np.cos(np.radians(targ_dec)), 
        Sdat_Dec - targ_dec)

        # cat source is a match if it is closest to targ and < 2" away
        targ_match = ( (source_sep < match_r/3600.) & 
            (source_sep == np.min(source_sep)) )
    
        if sum(targ_match) == 1:
            ZP = Sdat["ZEROPOINT"][targ_match][0]
            mag[imnum] = Sdat["MAG_AUTO"][targ_match][0] + ZP
            magunc[imnum] = Sdat["MAGERR_AUTO"][targ_match][0]
        else:
            mag[imnum] = -99.           # non-detection
            magunc[imnum] = -99.        # non-detection
    else:
        mag[imnum] = -99.           # non-detection
        magunc[imnum] = -99.        # non-detection

In [ ]:
targ_det = mag != -99.          # identify epochs where targ is detected

# plot mag vs HJD along with uncertainties
figure(figsize(13,8))
errorbar(HJD[targ_det], mag[targ_det], magunc[targ_det], fmt='ok')
ylim(np.max(mag[targ_det]) + 0.3, np.min(mag[targ_det]) - 0.3)
xlim(np.min(HJD[targ_det]) - 15, np.max(HJD[targ_det]) + 15)
xlabel(r'${\rm HJD}$', fontsize=18)
ylabel(r'$R \; {\rm mag}$', fontsize=18)

print "The LC scatter = %5.3f mag" % np.std(mag[targ_det])

## Links
* http://phares.caltech.edu/iptf/iptf_SummerSchool_2014/Miller2_problems.html